****Scraping data notebook****

**Installing and importing libraries**

In [ ]:
!pip install selenium

In [ ]:
!pip install tqdm

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [4]:
import imp
from lib2to3.pgen2 import driver
from sqlite3 import Time
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm import tqdm
import os
import requests

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


**Preparing desired data to scrape**

In [18]:
scrollnum = 100
sleepTimer = 3
chromeDriverPath = r"chromedriver.exe"

# choose the field of interest in scraping 
var = ['drink', 'soda', 'juice', 'drinkable', 'icecream', 'mokhito', 'smoothie', 'cocktail', 'tea', 'coke', 'liquor', 'mocktail', 'lemonade', 'fruit juice', 'coffee']



**Scraping data topic per topic**

Storing scraped data into list and removing profiles

In [ ]:
profiles = []
image_links = {}
for variable in var:
    print(variable)
    # url = f'https://www.pinterest.fr/search/pins/?q={variable}'
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option("excludeSwitches", ["enable-logging"])
    # options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox')
    # options.add_argument('--disable-dev-shm-usage')
    # driver = webdriver.Chrome(executable_path=chromeDriverPath,options=options)
    # driver.get(url)
    url = f'https://www.pinterest.fr/search/pins/?q={variable}'
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(executable_path=chromeDriverPath, options=options)
    driver.get(url)
    
    for _  in tqdm(range(1, scrollnum)):
        driver.execute_script("window.scrollTo(1,100000)")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')


        # find profiles pictures to put them out of the dataset
        listToStr = ' '.join([str(elem) for elem in soup.find_all("div", {"class": "Pj7 sLG XiG INd m1e"})])

        profile_pics = BeautifulSoup(listToStr, 'html.parser')
        for prof in profile_pics.findAll('img'):
            profiles.append(prof['src'])
            # print(prof['src'])


        # extract all images from the page
        
        for link in soup.findAll('img'):
            
            try:
              image_links[link['src']] = link['alt']
            except:
              pass
        
        sleep(sleepTimer)
    


**Storing scraped data into json file**

In [20]:
with open('data1.json', 'w') as fp:
    json.dump(image_links, fp)

In [21]:
for item in profiles:
    try:
        del image_links[item]
    except:
        pass


In [27]:
with open('data.json') as json_file:
    data = json.load(json_file)

**Storing scraped data into pandas dataframe**

In [32]:
df = pd.DataFrame(list(data.items()), columns = ['image_link','image_description'])
df.to_csv('scraped_data.csv')

In [12]:
os.mkdir('images')

In [35]:
df.head()

,image_link,image_description
0,https://i.pinimg.com/236x/ee/00/97/ee00970a205...,You Can Order A Bomb Pop Drink Off The Starbuc...
1,https://i.pinimg.com/236x/e9/30/32/e93032193ac...,3-Ingredient Korean Strawberry Milk
2,https://i.pinimg.com/236x/91/6a/86/916a86ae554...,root beer rum cocktail
3,https://i.pinimg.com/236x/98/19/52/9819524aabe...,This Salted Caramel Milkshake is made with cre...
4,https://i.pinimg.com/236x/80/e7/50/80e750de5ee...,Mermaid Slush - A fun and colorful kidMermaid ...


**Downloading data**

In [36]:
urls = list(data.keys())
for i in tqdm(range(len(urls)), desc = 'Downloading images'):
  response = requests.get(urls[i])
  filename = urls[i].split('/')[-1]

  if response.status_code == 200:
    with open('/content/images/' + filename, "wb") as image:
      for chunk in response.iter_content(chunk_size=128):
        image.write(chunk)

In [37]:
dataset = pd.DataFrame([(l.split('/')[-1], k) for l, k in data.items()], columns = ['id','description'])
dataset['index'] = range(1, len(dataset) + 1)
dataset.to_csv('dataset.csv')

**Compressing data to upload on drive (if necessary)**

In [39]:
!zip -r data.zip images dataset.csv

  adding: images/ (stored 0%)
  adding: images/cc0597a21784da3b2c8467dcf3985bcd.jpg (deflated 1%)
  adding: images/a74b50c1fafbd5f250ddf4b4f708efbe.jpg (deflated 5%)
  adding: images/d01b1958b691ab0e4038599b72f18e1d.jpg (deflated 1%)
  adding: images/1665fd109b44fbfdb4b6bffd570e21aa--tea-party-desserts-tea-recipes.jpg (deflated 0%)
  adding: images/3e6e40f6521d54f323638d3435a73c3b.jpg (deflated 1%)
  adding: images/b9afaa75932c3a1a105038dd940491ee--kefir-recipes-my-recipes.jpg (deflated 1%)
  adding: images/aea5f1974f17796acf1a148949a9357b.jpg (deflated 1%)
  adding: images/ac7012519fc86094c66ac7cdcbc091c5.jpg (deflated 3%)
  adding: images/035a6d4bced8bce6233129822cca15e6--juice-bar-kiosk-juice-bar-menu.jpg (deflated 0%)
  adding: images/432db96a75a05460860f9b5e60e981df.jpg (deflated 1%)
  adding: images/699a25b8b319fb6ceb121b605acda044.jpg (deflated 1%)
  adding: images/48c81bd7c52bd1db1ab054486a61436a--unicorn-food-unicorn-party.jpg (deflated 3%)
  adding: images/b2f68bc062e982bb40a